In [7]:
import cv2 as cv
import numpy as np
import torch

def mask2polygon(mask):
    mask=mask.astype("uint8")
    # mask=mask.astype("uint16")
    # mask=mask.astype(np.uint16)

    # show_NympyMask(mask)
    # print(f"mask.max() {mask.max()}")

    contours, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE) # im2, contours, hierarchy
    # print(f"contours len {len(contours)}")
    if len(contours)==0:
        return 0
    contours = contours[0]
    
    ### reduce amount point counturs
    peri = cv.arcLength(contours, True)
    eps = 0.006
    smother=eps*peri
    contours = cv.approxPolyDP(contours,smother, True)
    
    size = contours.shape[0]
    polygon = contours.reshape(size,2)
    
    return polygon

def polyg_relatived(polygon, resolution):
    polygon=np.float32(polygon)
    polygon[:,:1]=polygon[:,:1]/resolution[1]*100
    polygon[:,1:]=polygon[:,1:]/resolution[0]*100
    polygon=[xi.tolist() for xi in polygon]
    return polygon


##MY
def get_mask_byID(result, idx):
    # mask_np=results[0].masks.to().data[idx]
    mask_np=result.masks.to().data[idx]
    mask_np=mask_np.cpu().detach().numpy()
    return mask_np
def apply_mask(image, mask):
    """
    Cut image by mask
    """
    # Convert to numpy arrays
    image = np.array(image)
    mask = np.array(mask)
    # Convert grayscale image to RGB
    mask = np.stack((mask,)*3, axis=-1)
    # Multiply arrays
    resultant = image*mask
    return resultant


def resize_mask(mask_np, result):
    ## Get image size to resize mask back
    orig_h,orig_w=result.orig_shape
    size_orig=(orig_w,orig_h)

    ### Resize mask to initiale image size
    mask_np = cv2.resize(mask_np, size_orig, interpolation = cv2.INTER_AREA)
    return mask_np


def first_not_zero(zero_np):
    zero_np_sumed = zero_np.sum(axis=1) 
    zero_np_sumed
    return (zero_np_sumed !=0).argmax()

def last_not_zero(zero_np):
    zero_np_sumed = zero_np.sum(axis=1) 
    zero_np_sumed
    return np.max(np.nonzero(zero_np_sumed))

def slice_non_zero(masks):
    h,w=int(masks[0].shape[0]), int(masks[0].shape[1])
    zero=torch.zeros(h, w)
    for m in masks:
        zero=zero+m
    zero_np = zero.numpy()
    
    fnz=first_not_zero(zero_np)
    lnz=last_not_zero(zero_np)
    return (fnz,lnz)


def get_box_byID(result, idx):
    """
    Get box from yolo segmentation results
    """
    xyxy=result.boxes.xyxy[idx].cpu().detach().numpy()
    return xyxy.astype('int32')


def crop_box_xyxy(img, xyxy):
    """
    Crop image by xyxy bounding box
    """
    x1,y1,x2,y2=xyxy
    cropped = np.copy(img[y1:y2,  x1:x2,  :])
    return cropped


def basename_ext(pth_img):
    base_name = os.path.basename(pth_img)
    point = base_name.find(".")
    ext = base_name[point:]
    base_name = base_name[:point]
    
    return base_name, ext

def safly_create_dir(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [8]:
import os
import cv2
from ultralytics import YOLO
from PIL import Image

import matplotlib.pyplot as plt

class YoloSeg2cutted_image:
    def __init__(self, weight_model):
        self.model = YOLO(weight_model)
        self.classes = self.model.names

    def get_labels(self, pth_imgs, pth_out):
        
        for pth_img in os.listdir(pth_imgs):

            base_name, ext=basename_ext(pth_img)
            
            pth_img_ful = os.path.join(pth_imgs,pth_img)
            image = cv2.imread(pth_img_ful)
            h_orig,w_orig,_=image.shape
            resolution_label = (h_orig,w_orig)
            print(f"resolution_label {resolution_label}")

            # inference model # TODO be attentive about retina_masks=True 
            retina_masks=True 
            result=self.model(image, iou=0.35, conf=0.2, retina_masks=retina_masks )[0] # retina_masks=True

            # There are paddind, therefore we need to crop it
            if retina_masks==False: 
                fnz,lnz =slice_non_zero(result.masks.data) # # TODO uncoment insted prevuios string
                # fnz,lnz =  12,371 #  slice_non_zero(result.masks.data) # TODO check
                val_crop=(fnz,lnz)
                print(f"val_crop {val_crop}")
            

            amount_masks=len(result.masks.data)

            img_np= np.copy(result.orig_img)

            for idx_mask, cls in zip(range(amount_masks), result.boxes.cls):
                cls=int(cls)
                # val_crop attribute => crop slice because of padding
                mask_np=get_mask_byID(result, idx_mask)
                
                # TODO del because of padding
                if retina_masks==False:   # fnz,lnz = val_crop
                    mask_np = mask_np[fnz:lnz,:]

                mask_np = resize_mask(mask_np, result)

                ## Cut image by mask
                img_np_addMask=apply_mask(img_np, mask_np)

                ## Get box
                xyxy=get_box_byID(result,idx_mask)
                
                ## Crop by box
                img_np_cropBox=crop_box_xyxy(img_np_addMask,xyxy)
                w_crp,h_crp,_ = img_np_cropBox.shape
                if min(w_crp,h_crp) < 60:
                    continue

                class_box = str(self.classes[int(result.boxes.cls[idx_mask])])
                path_dir = os.path.join(pth_out, class_box)
                safly_create_dir(path_dir)
                pth_out_img = os.path.join(path_dir, f"{base_name}_{str(idx_mask)}{ext}") # {class_box}_
                cv2.imwrite(pth_out_img, img_np_cropBox)

            # print(f"result.orig_shape {result.orig_shape}")

        # return list_main_part

In [9]:
pth_imgs = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data/002_GO_PRO_SBER_20231101_ALL/selected_imgs"
pth_out = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data/002_GO_PRO_SBER_20231101_ALL/selected_imgs_crop/"

weight_model = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/003_train_leaf_tomato_15-10-23/models/weights/best.pt"

c = YoloSeg2cutted_image(weight_model)
c.get_labels(pth_imgs,pth_out)

resolution_label (1520, 2704)


0: 832x1440 112 leafs, 10 stems, 16 tomatos, 367.1ms
Speed: 7.9ms preprocess, 367.1ms inference, 544.6ms postprocess per image at shape (1, 3, 832, 1440)



resolution_label (1520, 2704)


0: 832x1440 130 leafs, 2 stems, 18 tomatos, 329.9ms
Speed: 5.9ms preprocess, 329.9ms inference, 1500.9ms postprocess per image at shape (1, 3, 832, 1440)



resolution_label (1520, 2704)


...

0: 832x1440 104 leafs, 8 stems, 28 tomatos, 355.2ms
Speed: 7.0ms preprocess, 355.2ms inference, 635.4ms postprocess per image at shape (1, 3, 832, 1440)



resolution_label (1520, 2704)


0: 832x1440 148 leafs, 6 stems, 20 tomatos, 337.3ms
Speed: 6.0ms preprocess, 337.3ms inference, 907.7ms postprocess per image at shape (1, 3, 832, 1440)



resolution_label (1520, 2704)


0: 832x1440 93 leafs, 15 stems, 94 tomatos, 374.1ms
Speed: 6.0ms preprocess, 374.1ms inference, 1897.5ms postprocess per image at shape (1, 3, 832, 1440)


In [ ]:
from ultralytics import YOLO

weight_model = "/Users/YaVolkonskiy/Documents/Projects/tomato/scripts/003_train_leaf_tomat_det_seg/best.pt"

model = YOLO(weight_model)

img = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data/002_GO_PRO_SBER_20231101_ALL/selected_imgs/GX010031_4.jpg"

model(img)[0].orig_img.shape

In [ ]:
h=1520
w=2704

min(h,w)